# COLLABORATIVE FILTERING USING THE NETFLIX DATA


In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1589214075031_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import Row
from pyspark.sql import *


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Problem 2: Analyzing the Netflix Data

Importing the Test Set from S3 bucket

In [3]:

lines = spark.read.text("s3://aws-emr-resources-446155163430-us-east-1/TestingRatings.txt").rdd
parts = lines.map(lambda row: row.value.split(","))
ratingsRDD = parts.map(lambda p: Row(MovieID=int(p[0]), CustomerId=int(p[1]),
                                     Rating=float(p[2])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Creating the Spark Dataframe

In [4]:
df = spark.createDataFrame(ratingsRDD)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(a) In order to be able to evaluate your approach the goal of your project is to predict the
ratings for all user-item pairs in the test set (TestingRatings.txt). How many distinct
items and how many distinct users are there in the test set(TestingRatings.txt)?

In [5]:
df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100478

In [6]:
df.select('CustomerId').distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27555

In [7]:
sc.install_pypi_package("pandas==1.0.3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [8]:
import numpy as np
import pandas as pd
Test = df.select("*").toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
Test.tail()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        CustomerId  MovieID  Rating
100473     1898310    17742     2.0
100474      716096    17742     4.0
100475       38115    17742     3.0
100476     2646347    17742     5.0
100477      273576    17742     2.0

Importing the Train Set from S3 Bucket

In [10]:

line = spark.read.text("s3://aws-emr-resources-446155163430-us-east-1/TrainingRatings.txt").rdd
part= line.map(lambda row: row.value.split(","))
rating = part.map(lambda p: Row(MovieID=int(p[0]), CustomerId=int(p[1]),
                                     Rating=float(p[2])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
Train = spark.createDataFrame(rating)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
train = Train.select("*").toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(b) The collaborative filtering approaches lives from finding many similar users (for a
user-user model) or many similar items (item-item model):
    
• user similarities are measured by overlappingitems

• item similarities are measured by overlappingusers

In [13]:
import pyspark.sql.functions as F
user_avg = Train.groupby('CustomerId')\
             .agg(F.avg('Rating').alias('mean_Rating'))\
             .orderBy(F.col('CustomerId')).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
user_avg.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+
|CustomerId|       mean_Rating|
+----------+------------------+
|         7|3.9038461538461537|
|        79| 3.630952380952381|
|       199| 3.943661971830986|
|       481| 4.351351351351352|
|       769| 3.193877551020408|
|       906|3.8333333333333335|
|      1310|3.4743589743589745|
|      1333|2.6734006734006734|
|      1427|3.5616438356164384|
|      1442| 4.362068965517241|
|      1457|3.5764705882352943|
|      1500|3.7111111111111112|
|      1527|3.6666666666666665|
|      1918|2.9220779220779223|
|      2000|3.6991150442477876|
|      2128|3.5294117647058822|
|      2213|3.9206349206349205|
|      2225|3.3563218390804597|
|      2307|3.7466666666666666|
|      2455| 3.409448818897638|
+----------+------------------+
only showing top 20 rows

In [15]:
user_avg.describe('mean_Rating').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|summary|        mean_Rating|
+-------+-------------------+
|  count|              28978|
|   mean|  3.516035221411006|
| stddev|0.43468284680471336|
|    min|                1.0|
|    max|                5.0|
+-------+-------------------+

In [16]:
def customFunction1(row):

    return (row.CustomerId,row.mean_Rating)

user = user_avg.rdd.map(customFunction1)
user.collect()[2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(199, 3.943661971830986)

In [17]:
type(user_avg)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [18]:
import pyspark.sql.functions as F
item_avg = Train.groupby('MovieID')\
             .agg(F.avg('Rating').alias('mean_Rating'))\
             .orderBy(F.col('MovieID')).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
item_avg.describe('mean_Rating').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|       mean_Rating|
+-------+------------------+
|  count|              1821|
|   mean| 3.058141964871255|
| stddev|0.5390238444452302|
|    min|1.4672897196261683|
|    max|               4.5|
+-------+------------------+

In [20]:
item_avg.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|MovieID|       mean_Rating|
+-------+------------------+
|      8| 3.055104203461674|
|     28| 3.760127409343352|
|     43| 2.310344827586207|
|     48|3.6206482593037217|
|     61|2.3859649122807016|
|     64| 2.466666666666667|
|     66|2.1384615384615384|
|     92|  3.08955223880597|
|     96|3.0136986301369864|
|    111|3.0373038930854155|
|    122|2.9027291812456264|
|    123|2.7641509433962264|
|    127| 2.626614261650758|
|    140|2.3902439024390243|
|    145|  2.99290780141844|
|    154|2.7586206896551726|
|    156| 3.354724757138652|
|    174| 2.899159663865546|
|    185| 2.976499690785405|
|    192|               3.0|
+-------+------------------+
only showing top 20 rows

User-User Approach:

In [21]:
pivot_Train = Train.groupBy("CustomerId").pivot("MovieID").sum("Rating").fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Item-Item Approach:

In [ ]:
pivot_train = Train.groupBy("MovieID").pivot("CustomerId").sum("Rating").fillna(0)

In [24]:
print ('Computing similarity matrix')
train2 = train.pivot(index = 'MovieID', columns = 'CustomerId',
                         values = 'Rating')
train2=train2.fillna(train2.mean(axis=0))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Computing similarity matrix

In [25]:
train3= train.pivot(index = 'CustomerId', columns = 'MovieID',
                         values = 'Rating')
train3=train3.fillna(train3.mean(axis=0))
user_mean = train2.mean(axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
rating_diff = train2.sub(train2.mean(axis=1), axis=0)
rating_diff 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CustomerId   7         79        199      ...   2649120   2649267   2649285
MovieID                                   ...                              
8           1.523898  0.154850  0.467560  ...  0.296050  0.375750 -0.495333
28          0.388328  0.019280  0.331990  ...  0.160480  0.240180  0.388328
43          0.389189  0.116295  0.429005  ...  0.257495  0.337195 -0.533888
48          0.377041  0.104148  0.416857  ...  0.245347  0.325047 -0.546035
61          0.388908  0.116014  0.428724  ...  0.257214  0.336914 -0.534169
...              ...       ...       ...  ...       ...       ...       ...
17725       0.381802  0.108908  0.421618  ...  0.250108  0.329808 -0.541275
17728       0.389000  0.116106  0.428816  ...  0.257306  0.337006 -0.534077
17734       0.388153  0.115259  0.427969  ...  0.256459  0.336159 -0.534924
17741       0.395723  0.122829  0.435539  ...  0.264028  0.343728 -0.527354
17742       0.392827  0.119933  0.432643  ...  0.261133  0.340833 -0.530250

[1821 rows 

In [27]:
cor_matrix = train3.corr()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
cor_matrix

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MovieID     8         28        43     ...     17734     17741     17742
MovieID                                ...                              
8        1.000000  0.026173  0.014027  ...  0.041982  0.018298  0.003809
28       0.026173  1.000000  0.013081  ...  0.026872  0.035921  0.003725
43       0.014027  0.013081  1.000000  ...  0.067302  0.044276  0.036744
48       0.017317  0.081754  0.022921  ...  0.052179  0.046530  0.005936
61       0.014884  0.004485  0.048389  ...  0.042341  0.034853  0.040344
...           ...       ...       ...  ...       ...       ...       ...
17725    0.016143  0.010176  0.051562  ...  0.053199  0.036778  0.013022
17728    0.017812  0.007724  0.049892  ...  0.059174  0.010200  0.050991
17734    0.041982  0.026872  0.067302  ...  1.000000  0.065275  0.010723
17741    0.018298  0.035921  0.044276  ...  0.065275  1.000000  0.004309
17742    0.003809  0.003725  0.036744  ...  0.010723  0.004309  1.000000

[1821 rows x 1821 columns]

In [33]:

similar=cor_matrix.unstack().sort_values(ascending=False).drop_duplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

k-most Similar Items

In [34]:
similar

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MovieID  MovieID
17742    17742      1.000000
11319    14470      0.707351
16904    6714       0.707305
5898     12946      0.694443
4868     7334       0.689057
                      ...   
6991     16576     -0.243432
11838    13769     -0.268429
17232    11838     -0.273308
8933     6277      -0.280256
6277     11838     -0.306602
Length: 1655404, dtype: float64

# Problem 1: Collaborative Filtering Approach

Similarity Measure Using Correlation

In [35]:
from pyspark.mllib.stat import Statistics
import pandas as pd

# result can be used w/ seaborn's heatmap
def compute_correlation_matrix(df, method='pearson'):
    # wrapper around
    # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
    df_rdd = df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method=method)
    corr_mat_df = pd.DataFrame(corr_mat,
                   columns=df.columns, 
                    index=df.columns)
    return corr_mat_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
corr=compute_correlation_matrix(pivot_Train, method='pearson')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

K-similar users:

In [41]:

similar_Users=corr.unstack().sort_values(ascending=False).drop_duplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
similar_Users

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17742  17742    1.000000
7334   4868     0.863401
14283  1256     0.829364
7903   11088    0.823021
12946  5898     0.812781
                  ...   
1046   10774   -0.244882
10774  16082   -0.246363
3638   6336    -0.246522
15582  10774   -0.271616
10774  13887   -0.278632
Length: 1658666, dtype: float64

In [42]:
correlation = spark.createDataFrame(corr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
def customFunction(row):

    return (row.CustomerId,row.MovieID,row.Rating)

sample2 = Train.rdd.map(customFunction)
sample2.collect()[2][1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8

In [44]:
def customFunction1(row):

    return (row.MovieID,row.mean_Rating)

sample = item_avg.rdd.map(customFunction1)
sample.collect()[2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(43, 2.310344827586207)

In [45]:
def customFunctionuser(row):

    return (row.MovieID,row.mean_Rating)

sample_user = item_avg.rdd.map(customFunction1)
sample_user.collect()[2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(43, 2.310344827586207)

In [46]:
def customFunction2(row):

    return (row.MovieID)

user_sample = item_avg.rdd.map(customFunction2)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Problem 3: Collaborative Filtering Implementation

Item-Item Based Approach

Step 1: Implementation

In [47]:
print ('Building predictions')
item_mean_of_means = 3.058141964871257
items_all = user_sample.collect()
movies_all = rating_diff.columns
predictions = []

for i in range(df.count()):
        
        movie_k = sample2.collect()[i][0]
        item_i =sample2.collect()[i][1]
        # Get item i's mean or assign mean of means
        if item_i in items_all:
            item_i_mean = sample_user.collect()[i][0]
        else:
            item_i_mean = item_mean_of_means
        # Obtain all ratings of movie k
        if movie_k in movies_all:
            rating_diff_js = rating_diff.loc[:,movie_k]
            users_j_no_k = rating_diff_js[rating_diff_js.isnull()].index
            # Obtain similarities of item i and items that has same user
            w = cor_matrix.loc[user_i,:].copy()
            w.loc[users_j_no_k] = None
            sum_w = w.abs().sum()
            # Calculate collaborative rating
            if sum_w != 0:
                coll_rating = (w * rating_diff_js).sum() / sum_w
            else:
                coll_rating = 0
        else:
            coll_rating = 0
        rating_ik = user_i_mean + coll_rating
        # Check bound and round to nearest rating
        predictions.append( round(max(min(rating_ik, 5),1),1) )

       

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Building predictions

Step 2: Execution and Evaluation

In [48]:
print( 'Summary results:')
Test['predictions'] = predictions
print ('  RMSE:' , round(np.sqrt( ((Test['Rating'] -
                                     Test['predictions'])**2).mean()),4))
print ('  MAE: ' , round(np.mean(abs(Test['Rating'] -
        Test['predictions'])),4), '\n')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Summary results:
  RMSE: 0.9438
  MAE:  0.7485

Step 3: Does your approach work for your own preferences?

In [49]:

columns = ['CustomerId', 'MovieID', 'Rating']
vals = [(167234,140 , 3.0), (167234,123 , 2.0),(167234,154 , 2.0),(167234,185 , 1.0),]

newRow = spark.createDataFrame([(167234,140 , 3.0), (167234,123 , 4.0),(167234,154 , 2.0),(167234,185 , 1.0)], columns)
appended = df.union(newRow)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
Created = appended.select("*").toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
print ('Building predictions')
item_mean_of_means = 3.058141964871257
items_all = user_sample.collect()
movies_all = rating_diff.columns
predictions = []

for i in range(appended.count()):
        
        movie_k = sample2.collect()[i][0]
        item_i =sample2.collect()[i][1]
        # Get item i's mean or assign mean of means
        if item_i in items_all:
            item_i_mean = sample_user.collect()[i][0]
        else:
            item_i_mean = item_mean_of_means
        # Obtain all ratings of movie k
        if movie_k in movies_all:
            rating_diff_js = rating_diff.loc[:,movie_k]
            users_j_no_k = rating_diff_js[rating_diff_js.isnull()].index
            # Obtain similarities of item i and items that has same user
            w = cor_matrix.loc[user_i,:].copy()
            w.loc[users_j_no_k] = None
            sum_w = w.abs().sum()
            # Calculate collaborative rating
            if sum_w != 0:
                coll_rating = (w * rating_diff_js).sum() / sum_w
            else:
                coll_rating = 0
        else:
            coll_rating = 0
        rating_ik = user_i_mean + coll_rating
        # Check bound and round to nearest rating
        predictions.append( round(max(min(rating_ik, 5),1),1) )

       

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Building predictions

In [52]:
Created['predictions'] = predictions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The Predictions of the Created new User with Four movies are Predicted pretty well with the above model.